In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
gpt_data=pd.read_csv('/content/Feedback on ChatGPT.csv')
gpt_data.columns

Index(['Timestamp', '1. Your good name', '2. How often do you use ChatGPT?',
       '3. What do you typically use ChatGPT for?',
       '4. Share your opinion on ChatGPT',
       '5. On a scale of 1-10, how easy is it to navigate and use to ChatGPT?',
       '6. Do you think ChatGPT accurately understands your questions and provides relevant responses?',
       'Which age group do you fall under?',
       '7. Would you prefer solving an assignment or a simple programming question using ChatGPT even when you have ample time to submit the assignment?',
       '8. Do you think ChatGPT destroys people's ability to think and makes them lazy?'],
      dtype='object')

In [ ]:
ds = pd.DataFrame(gpt_data.iloc[:,4])
ds.columns = ['review']
ds

,review
0,Very useful
1,9
2,Mostly yes
3,Not much satisfied.
4,Very much satisfied
...,...
258,Good
259,Very good
260,Very helpful
261,It is helpful but spoils thinking


In [ ]:
# Define a function to clean the text
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text



# Cleaning the text in the review column
ds['Cleaned Reviews'] = ds['review'].fillna('').apply(clean)
ds.head()

,review,Cleaned Reviews
0,Very useful,Very useful
1,9,
2,Mostly yes,Mostly yes
3,Not much satisfied.,Not much satisfied
4,Very much satisfied,Very much satisfied


In [ ]:
ds = ds.dropna(axis= 0)
ds.head

<bound method NDFrame.head of                                                 review  \
0                                          Very useful   
1                                                    9   
2                                           Mostly yes   
3                                  Not much satisfied.   
4                                 Very much satisfied    
..                                                 ...   
258                                               Good   
259                                          Very good   
260                                       Very helpful   
261                 It is helpful but spoils thinking    
262  good and time saving sometimes. would be bette...   

                                       Cleaned Reviews  
0                                          Very useful  
1                                                       
2                                           Mostly yes  
3                                  Not much s

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
          newlist.append(tuple([word, pos_dict.get(tag[0])]))
        return newlist

ds['POS tagged'] = ds['Cleaned Reviews'].fillna('').apply(token_stop_pos)
ds.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
<ipython-input-8-aab5ea0ff777>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['POS tagged'] = ds['Cleaned Reviews'].fillna('').apply(token_stop_pos)


,review,Cleaned Reviews,POS tagged
0,Very useful,Very useful,[]
1,9,,None
2,Mostly yes,Mostly yes,"[(Mostly, r)]"
3,Not much satisfied.,Not much satisfied,[]
4,Very much satisfied,Very much satisfied,[]


In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
      if not pos:
          lemma = word
          lemma_rew = lemma_rew + " " + lemma
      else:
          lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
          lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

ds['Lemma'] = ds['POS tagged'].fillna('').apply(lemmatize)
ds.head()

<ipython-input-9-69624fa5ccbc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['Lemma'] = ds['POS tagged'].fillna('').apply(lemmatize)


,review,Cleaned Reviews,POS tagged,Lemma
0,Very useful,Very useful,[],
1,9,,None,
2,Mostly yes,Mostly yes,"[(Mostly, r)]",Mostly
3,Not much satisfied.,Not much satisfied,[],
4,Very much satisfied,Very much satisfied,[],


In [ ]:
from textblob import TextBlob
# function to calculate subjectivity
def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity
# function to calculate polarity
def getPolarity(review):
    return TextBlob(review).sentiment.polarity

# function to analyze the reviews
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

fin_data = pd.DataFrame(ds[['review', 'Lemma']])
# fin_data['Subjectivity'] = fin_data['Lemma'].apply(getSubjectivity) 
fin_data['Polarity'] = fin_data['Lemma'].apply(getPolarity) 
fin_data['Analysis'] = fin_data['Polarity'].apply(analysis)


In [ ]:
train_x, test_x, train_y, test_y = train_test_split(fin_data['Lemma'], fin_data['Analysis'], test_size = 0.3)

Encoder = LabelEncoder()
train_y = Encoder.fit_transform(train_y)
test_y = Encoder.fit_transform(test_y)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(fin_data['Lemma'])

train_x_tfidf = Tfidf_vect.transform(train_x)
test_x_tfidf = Tfidf_vect.transform(test_x)

SVM = SVC(C=1.0, kernel = 'linear', degree = 3, gamma = 'auto')
SVM.fit(train_x_tfidf, train_y)

predictions_SVM = SVM.predict(test_x_tfidf)


In [ ]:
from sklearn.metrics import confusion_matrix
target_names = ['Negative', 'Neutral', 'Positive']
print(classification_report(test_y, predictions_SVM, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         2
     Neutral       0.88      1.00      0.93        50
    Positive       1.00      0.81      0.90        27

    accuracy                           0.91        79
   macro avg       0.63      0.60      0.61        79
weighted avg       0.90      0.91      0.90        79



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_x_tfidf,train_y)
y_pred_lr = lr.predict(test_x_tfidf)
print(classification_report(test_y, y_pred_lr, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         2
     Neutral       0.86      1.00      0.93        50
    Positive       1.00      0.78      0.88        27

    accuracy                           0.90        79
   macro avg       0.62      0.59      0.60        79
weighted avg       0.89      0.90      0.89        79



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.naive_bayes import GaussianNB
gb = GaussianNB()
gb.fit(train_x_tfidf.toarray(),train_y)
y_pred_gb = gb.predict(test_x_tfidf.toarray())
print(classification_report(test_y, y_pred_gb, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.04      1.00      0.08         2
     Neutral       1.00      0.18      0.31        50
    Positive       1.00      0.81      0.90        27

    accuracy                           0.42        79
   macro avg       0.68      0.66      0.43        79
weighted avg       0.98      0.42      0.50        79



In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth =3, random_state = 42)
clf.fit(train_x_tfidf.toarray(),train_y)
y_pred_clf = clf.predict(test_x_tfidf.toarray())
print(classification_report(test_y, y_pred_clf, target_names=target_names))


              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         2
     Neutral       0.83      1.00      0.91        50
    Positive       1.00      0.70      0.83        27

    accuracy                           0.87        79
   macro avg       0.61      0.57      0.58        79
weighted avg       0.87      0.87      0.86        79



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(train_x_tfidf.toarray(),train_y)
y_pred_knn = knn.predict(test_x_tfidf.toarray())
print(classification_report(test_y, y_pred_knn, target_names=target_names))


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         2
     Neutral       0.83      1.00      0.91        50
    Positive       1.00      0.70      0.83        27

    accuracy                           0.87        79
   macro avg       0.61      0.57      0.58        79
weighted avg       0.87      0.87      0.86        79



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

rfc = RandomForestClassifier(max_depth=2, random_state=0)
rfc.fit(train_x_tfidf.toarray(),train_y)
RandomForestClassifier(...)
print(classification_report(test_y, y_pred_knn, target_names=target_names))




              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         2
     Neutral       0.83      1.00      0.91        50
    Positive       1.00      0.70      0.83        27

    accuracy                           0.87        79
   macro avg       0.61      0.57      0.58        79
weighted avg       0.87      0.87      0.86        79



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.linear_model import LinearRegression
regr = LinearRegression()
 
regr.fit(train_x_tfidf.toarray(),train_y)
print(classification_report(test_y, y_pred_knn, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         2
     Neutral       0.83      1.00      0.91        50
    Positive       1.00      0.70      0.83        27

    accuracy                           0.87        79
   macro avg       0.61      0.57      0.58        79
weighted avg       0.87      0.87      0.86        79



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


import numpy as np
from sklearn.metrics import mean_squared_error

class logistic_regr:

    def __init__(self):
        from sklearn.linear_model import LogisticRegression
        self.model = LogisticRegression(penalty="l2",solver='liblinear')
        self.flag = False
        self.flag_y_pred = None

    def fit(self, X, y):
        y_unique = list(set(y))
        if len(y_unique) == 1:
            self.flag = True
            self.flag_y_pred = y_unique[0]
        else:
            self.model.fit(X, y)

    def predict(self, X):
        if self.flag:
            return self.flag_y_pred * np.ones((len(X),), dtype=int)
        else:
            return self.model.predict(X)

    def loss(self, X, y, y_pred):
        return mean_squared_error(y, y_pred)

    def predict_proba(self,X):
        return self.model.predict_proba(X)


esitmators=5
y_pred=[]
n_train_split=int(len(train_x)/esitmators)
inital_train=0
final_train=0


yy_pred=[]
classifier=None

for i in range(1,esitmators+1):

    classifier =logistic_regr()

    final_train=i*n_train_split
    temp_X_train=train_x_tfidf.toarray()[inital_train:final_train]
    temp_y_train=train_y[inital_train:final_train]


    L=ModelTree(classifier,max_depth=20, min_samples_leaf=10)

    node=L.fit(temp_X_train,temp_y_train,verbose=False)
    classifier=node["model"]

    y_pred_temp=L.predict(test_x)
    yy_pred.append(y_pred_temp)

for j in range(len(yy_pred[0])):
    curr=[]
    for i in range(len(yy_pred)):
        curr.append(yy_pred[i][j])
    a=curr.count(0)
    b=curr.count(1)
    if a>b:
        y_pred.append(0)
    else:
        y_pred.append(1)




NameError: ignored